In [2]:
!pip install yfinance

In [3]:
import pandas_datareader
import pandas_datareader.data as reader
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import statsmodels.api as sm
from scipy.optimize import minimize
import yfinance as yfin
import seaborn as sns
import plotly.express as px
import warnings
import numpy as np
warnings.filterwarnings('ignore')
yfin.pdr_override()

/usr/local/lib/python3.10/dist-packages/yfinance/base.py:48: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  _empty_series = pd.Series()


In [4]:
start = '2000-01-01'
end = '2023-11-18'
brk_data = reader.get_data_yahoo('BRK-A', start, end)['Adj Close']
print(brk_data)

[*********************100%%**********************]  1 of 1 completed

Date
2000-01-03     54800.0
2000-01-04     52000.0
2000-01-05     53200.0
2000-01-06     55000.0
2000-01-07     56500.0
                ...   
2023-11-13    531288.0
2023-11-14    537400.0
2023-11-15    542500.0
2023-11-16    546635.0
2023-11-17    544190.0
Name: Adj Close, Length: 6009, dtype: float64


In [5]:
factors_data = pandas_datareader.famafrench.FamaFrenchReader('F-F_Research_Data_Factors_daily',start,end).read()[0]
factors = factors_data[['Mkt-RF', 'HML', 'SMB']]

In [6]:
merged_data = pd.merge(brk_data, factors, left_index=True, right_index=True).dropna(axis=0)
y = merged_data['Adj Close'].pct_change() - factors_data['RF']/100
y = y.dropna(axis=0)
print(y)

Date
2000-01-04   -0.051305
2000-01-05    0.022867
2000-01-06    0.033625
2000-01-07    0.027063
2000-01-10   -0.009060
                ...   
2023-11-13   -0.001334
2023-11-14    0.011294
2023-11-15    0.009280
2023-11-16    0.007412
2023-11-17   -0.004683
Length: 6008, dtype: float64


In [7]:

start = '2000-01-04'
factors_data = pandas_datareader.famafrench.FamaFrenchReader('F-F_Research_Data_Factors_daily',start,end).read()[0]
factors = factors_data[['Mkt-RF', 'HML', 'SMB']]
X = sm.add_constant(factors)
print(X)

            const  Mkt-RF   HML   SMB
Date                                 
2000-01-04    1.0   -4.06  2.06  0.00
2000-01-05    1.0   -0.09  0.16  0.20
2000-01-06    1.0   -0.73  1.26 -0.43
2000-01-07    1.0    3.21 -1.42 -0.48
2000-01-10    1.0    1.76 -1.52  0.96
...           ...     ...   ...   ...
2023-11-13    1.0   -0.05 -0.27  0.15
2023-11-14    1.0    2.18  1.18  2.99
2023-11-15    1.0    0.18  0.49 -0.04
2023-11-16    1.0   -0.11 -0.31 -1.40
2023-11-17    1.0    0.25  0.62  0.99

[6008 rows x 4 columns]


In [8]:
model = sm.OLS(y, X)
results = model.fit()

print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.387
Model:                            OLS   Adj. R-squared:                  0.386
Method:                 Least Squares   F-statistic:                     1262.
Date:                Fri, 26 Jan 2024   Prob (F-statistic):               0.00
Time:                        07:45:10   Log-Likelihood:                 18586.
No. Observations:                6008   AIC:                        -3.716e+04
Df Residuals:                    6004   BIC:                        -3.714e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0002      0.000      1.413      0.1

In [9]:
alphas = []
alpha = results.params[0]
alphas.append(alpha)
buffet_alpha = np.mean(alphas)
is_significant = results.pvalues['const'] < 0.05
print("Buffett's Alpha: ", buffet_alpha)
print("Is the result statistically significant? ", is_significant)

Buffett's Alpha:  0.00020011816222520012
Is the result statistically significant?  False


Акции Berkshire Pathways (BRK-A) демонстрируют положительную альфу, что указывает на то, что они превзошли доходность, скорректированную на факторы Фама-Френч.